In [1]:
import circle_cal.model as ccm
from workalendar.usa import UnitedStates, Indiana
from workalendar.core import ChristianMixin
from datetime import datetime, date


In [2]:
year = ccm.Year(2024)

In [8]:
year.to_theta(year.start.datetime())

0.0

In [32]:
def NotreDame(Indiana):
    include_easter_monday=True
    include_good_friday=True
    include_easter_sunday=True
    include_christmas_eve=True

In [14]:
cal = Notre_Dame()
cal.holidays()

[(datetime.date(2024, 1, 1), 'New year'),
 (datetime.date(2024, 1, 15), 'Birthday of Martin Luther King, Jr.'),
 (datetime.date(2024, 2, 14), 'Ash Wednesday'),
 (datetime.date(2024, 2, 19), "Washington's Birthday"),
 (datetime.date(2024, 5, 27), 'Memorial Day'),
 (datetime.date(2024, 7, 4), 'Independence Day'),
 (datetime.date(2024, 9, 2), 'Labor Day'),
 (datetime.date(2024, 10, 14), 'Columbus Day'),
 (datetime.date(2024, 11, 11), 'Veterans Day'),
 (datetime.date(2024, 11, 28), 'Thanksgiving Day'),
 (datetime.date(2024, 12, 25), 'Christmas Day')]

In [15]:
cal = UnitedStates(include_ash_wednesday=True)

In [21]:
cal.include_ash_wednesday = True

In [25]:
cal.get_calendar_holidays(2024)

[(datetime.date(2024, 1, 1), 'New year'),
 (datetime.date(2024, 7, 4), 'Independence Day'),
 (datetime.date(2024, 11, 11), 'Veterans Day'),
 (datetime.date(2024, 2, 14), 'Ash Wednesday'),
 (datetime.date(2024, 12, 25), 'Christmas Day'),
 (datetime.date(2024, 1, 15), 'Birthday of Martin Luther King, Jr.'),
 (datetime.date(2024, 5, 27), 'Memorial Day'),
 (datetime.date(2024, 9, 2), 'Labor Day'),
 (datetime.date(2024, 11, 28), 'Thanksgiving Day'),
 (datetime.date(2024, 2, 19), "Washington's Birthday"),
 (datetime.date(2024, 10, 14), 'Columbus Day')]

In [41]:
import calendar

In [42]:
calendar

'                                  2024\n\n      January                   February                   March\nMo Tu We Th Fr Sa Su      Mo Tu We Th Fr Sa Su      Mo Tu We Th Fr Sa Su\n 1  2  3  4  5  6  7                1  2  3  4                   1  2  3\n 8  9 10 11 12 13 14       5  6  7  8  9 10 11       4  5  6  7  8  9 10\n15 16 17 18 19 20 21      12 13 14 15 16 17 18      11 12 13 14 15 16 17\n22 23 24 25 26 27 28      19 20 21 22 23 24 25      18 19 20 21 22 23 24\n29 30 31                  26 27 28 29               25 26 27 28 29 30 31\n\n       April                      May                       June\nMo Tu We Th Fr Sa Su      Mo Tu We Th Fr Sa Su      Mo Tu We Th Fr Sa Su\n 1  2  3  4  5  6  7             1  2  3  4  5                      1  2\n 8  9 10 11 12 13 14       6  7  8  9 10 11 12       3  4  5  6  7  8  9\n15 16 17 18 19 20 21      13 14 15 16 17 18 19      10 11 12 13 14 15 16\n22 23 24 25 26 27 28      20 21 22 23 24 25 26      17 18 19 20 21 22 23\n29 30    

In [45]:
cal.holidays()

[(datetime.date(2024, 1, 1), 'New year'),
 (datetime.date(2024, 1, 15), 'Birthday of Martin Luther King, Jr.'),
 (datetime.date(2024, 2, 19), "Washington's Birthday"),
 (datetime.date(2024, 5, 27), 'Memorial Day'),
 (datetime.date(2024, 7, 4), 'Independence Day'),
 (datetime.date(2024, 9, 2), 'Labor Day'),
 (datetime.date(2024, 10, 14), 'Columbus Day'),
 (datetime.date(2024, 11, 11), 'Veterans Day'),
 (datetime.date(2024, 11, 28), 'Thanksgiving Day'),
 (datetime.date(2024, 12, 25), 'Christmas Day')]